In [1]:
from yugiquery import *
header('Speed Duel')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Speed Duel - Execution started 15/02/2023 10:50 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

# Data aquisition

## Fetch online data

In [11]:
# Timestamp
timestamp = pd.Timestamp.now()

In [2]:
# Fetch skill cards
skill_df = fetch_skill()

# Fetch deck cards
speed_df = fetch_speed()

105 results



## Merge data

In [13]:
full_df = pd.concat([speed_df,skill_df], ignore_index=True, axis=0).sort_values('Name').reset_index(drop=True)
print('Data merged')

,Name,Password,Card type,Property,Primary type,Secondary type,Attribute,Monster type,Level/Rank,DEF,Effect type,Archseries,ATK,TCG status,OCG status,TCG Speed Duel status,Artwork,Page name,Page URL,Modification date,Character
0,7 Completed,86198326,Spell Card,Equip Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Condition, Continuous-like Effect)",NaN,NaN,Unlimited,Unlimited,Unlimited,NaN,7 Completed,https://yugipedia.com/wiki/7_Completed,2021-10-31 17:18:50,NaN
1,Absorbing Jar,03900605,Monster Card,NaN,Effect Monster,"(Flip monster,)",EARTH,Rock,3,500,"(Flip effect,)","(Jar,)",600,Unlimited,Unlimited,Unlimited,NaN,Absorbing Jar,https://yugipedia.com/wiki/Absorbing_Jar,2021-11-06 15:22:32,NaN
2,Acid Trap Hole,41356845,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)","(Trap Hole (archetype),)",NaN,Unlimited,Unlimited,Unlimited,"(Alternate,)",Acid Trap Hole,https://yugipedia.com/wiki/Acid_Trap_Hole,2023-01-20 13:48:24,NaN
3,Adhesion Trap Hole,62325062,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Activation condition, Effect)","(Trap Hole (archetype),)",NaN,Unlimited,Unlimited,Unlimited,NaN,Adhesion Trap Hole,https://yugipedia.com/wiki/Adhesion_Trap_Hole,2022-08-27 15:04:42,NaN
4,Advanced Dark,12644061,Spell Card,Field Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Continuous-like Effect, Trigger Effect)",NaN,NaN,Unlimited,Unlimited,Unlimited,NaN,Advanced Dark,https://yugipedia.com/wiki/Advanced_Dark,2022-12-01 14:56:47,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,Union Combination,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,NaN,Union Combination,https://yugipedia.com/wiki/Union_Combination,2023-02-09 12:07:11,Seto Kaiba
993,Viral Infection,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,NaN,Viral Infection,https://yugipedia.com/wiki/Viral_Infection,2020-05-29 17:27:57,Seto Kaiba
994,Volcanic Cannon,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Volcanic,)",NaN,Illegal,NaN,Legal,NaN,Volcanic Cannon,https://yugipedia.com/wiki/Volcanic_Cannon,2022-09-01 17:16:50,Axel Brodie
995,The World's Greatest Fisherman,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,NaN,The World's Greatest Fisherman,https://yugipedia.com/wiki/The_World%27s_Great...,2020-05-29 17:49:58,Mako Tsunami


## Save data

In [17]:
full_df.to_csv(f'../data/all_speed_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
print('Data saved')

# Check changes

## Load previous data

In [18]:
# Get list of files
files_list = sorted(glob.glob('../data/all_speed_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if len(files_list)>1:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[1], dtype=object, keep_default_na=False, na_values=[''])
    # Correct tuples
    previous_df['Primary type'] = previous_df['Secondary type'].dropna().apply(literal_eval)
    previous_df['Effect type'] = previous_df['Effect type'].dropna().apply(literal_eval)
    previous_df['Link Arrows'] = previous_df['Link Arrows'].dropna().apply(literal_eval)
    previous_df['Archseries'] = previous_df['Archseries'].dropna().apply(literal_eval)
    previous_df['Artwork'] = previous_df['Artwork'].dropna().apply(literal_eval)
    previous_df['Errata'] = previous_df['Errata'].dropna().apply(literal_eval)
    # Force dtypes to match current df
    previous_df = previous_df.astype(full_df[previous_df.columns].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[1]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

No older files


## Generate changelog

In [ ]:
if previous_df is None:
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, full_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/speed_changelog_{timestamp.isoformat(timespec="minutes")}_{previous_ts.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

# Data visualization

In [14]:
full_df

,Name,Password,Card type,Property,Primary type,Secondary type,Attribute,Monster type,Level/Rank,DEF,Effect type,Archseries,ATK,TCG status,OCG status,TCG Speed Duel status,Artwork,Page name,Page URL,Modification date,Character
0,7 Completed,86198326,Spell Card,Equip Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Condition, Continuous-like Effect)",NaN,NaN,Unlimited,Unlimited,Unlimited,NaN,7 Completed,https://yugipedia.com/wiki/7_Completed,2021-10-31 17:18:50,NaN
1,Absorbing Jar,03900605,Monster Card,NaN,Effect Monster,"(Flip monster,)",EARTH,Rock,3,500,"(Flip effect,)","(Jar,)",600,Unlimited,Unlimited,Unlimited,NaN,Absorbing Jar,https://yugipedia.com/wiki/Absorbing_Jar,2021-11-06 15:22:32,NaN
2,Acid Trap Hole,41356845,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)","(Trap Hole (archetype),)",NaN,Unlimited,Unlimited,Unlimited,"(Alternate,)",Acid Trap Hole,https://yugipedia.com/wiki/Acid_Trap_Hole,2023-01-20 13:48:24,NaN
3,Adhesion Trap Hole,62325062,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Activation condition, Effect)","(Trap Hole (archetype),)",NaN,Unlimited,Unlimited,Unlimited,NaN,Adhesion Trap Hole,https://yugipedia.com/wiki/Adhesion_Trap_Hole,2022-08-27 15:04:42,NaN
4,Advanced Dark,12644061,Spell Card,Field Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,"(Continuous-like Effect, Trigger Effect)",NaN,NaN,Unlimited,Unlimited,Unlimited,NaN,Advanced Dark,https://yugipedia.com/wiki/Advanced_Dark,2022-12-01 14:56:47,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,Union Combination,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,NaN,Union Combination,https://yugipedia.com/wiki/Union_Combination,2023-02-09 12:07:11,Seto Kaiba
993,Viral Infection,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,NaN,Viral Infection,https://yugipedia.com/wiki/Viral_Infection,2020-05-29 17:27:57,Seto Kaiba
994,Volcanic Cannon,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Volcanic,)",NaN,Illegal,NaN,Legal,NaN,Volcanic Cannon,https://yugipedia.com/wiki/Volcanic_Cannon,2022-09-01 17:16:50,Axel Brodie
995,The World's Greatest Fisherman,NaN,Skill Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Illegal,NaN,Legal,NaN,The World's Greatest Fisherman,https://yugipedia.com/wiki/The_World%27s_Great...,2020-05-29 17:49:58,Mako Tsunami


## Property

In [6]:
full_df['Property'].nunique()

3

In [5]:
full_df['Property'].value_counts()

Field Spell Card         10
Continuous Spell Card     2
Continuous Trap Card      1
Name: Property, dtype: int64

## Archseries

In [9]:
full_df['Archseries'].explode().nunique()

16

In [11]:
full_df['Archseries'].explode().value_counts()

Fusion (archetype)               3
Crystal                          2
Cyber                            2
HERO                             2
Toon                             2
roid                             2
Guardian                         1
Blaze Accelerator (archetype)    1
Cyberdark                        1
Duel winner                      1
Harpie                           1
Gravekeeper's                    1
Ojama                            1
Machine Angel                    1
Signature move                   1
Volcanic                         1
Name: Archseries, dtype: int64

## TCG Speed Duel status

In [13]:
full_df['TCG Speed Duel status'].nunique()

1

In [12]:
full_df['TCG Speed Duel status'].value_counts()

Legal    103
Name: TCG Speed Duel status, dtype: int64

## TCG status

In [14]:
full_df['TCG status'].nunique()

1

In [15]:
full_df['TCG status'].value_counts()

Illegal    103
Name: TCG status, dtype: int64

## Character

In [7]:
skill_df['Character'].nunique()

36

In [8]:
skill_df['Character'].value_counts()

Yami Yugi              10
Seto Kaiba              6
Joey Wheeler            5
Bonz                    5
Maximillion Pegasus     5
Ishizu Ishtar           5
Jaden Yuki              4
Rex Raptor              4
Syrus Truesdale         4
Bandit Keith            4
Mai Valentine           4
Mako Tsunami            4
Paradox Brothers        3
Bastion Misawa          3
Weevil Underwood        3
Yami Marik              3
Aster Phoenix           2
Espa Roba               2
Yami Bakura             2
Vellian Crowler         2
Odion                   2
Chazz Princeton         2
Zane Truesdale          2
Alexis Rhodes           2
Jesse Anderson          2
Axel Brodie             2
Christine               1
David                   1
Tyranno Hassleberry     1
Yubel                   1
Chumley Huffington      1
Andrew                  1
Arkana                  1
Téa Gardner             1
Emma (Duel Links)       1
Lumis and Umbra         1
Name: Character, dtype: int64

In [4]:
footer()

<div align='right'>
    Execution completed 11/02/2023 17:49 UTC
</div>

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Speed.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Ss]peed*"

In [ ]:
! git commit -m {"'Speed duel update-" + timestamp.isoformat() + "'"}